In [93]:
import numpy as np
from numba import njit, jit
import pickle
from numba.typed import List

from scipy.spatial.distance import cdist, pdist

In [177]:
var = pickle.load(open('var1.pickle', 'rb'))

In [178]:
Box = var ['BOX']
BOX_miu = var ['BOX_miu']
BOX_S = var ['BOX_S']
NB = var ['NB']
intervel1 = var ['interval1']
intervel2 = var ['interval2']
print(type(Box))
print(type(BOX_miu))
print(type(BOX_S))
print(type(NB))
print(type(intervel1))
print(type(intervel2))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.float64'>
<class 'numpy.float64'>


In [179]:
print(Box)
print(BOX_miu)
print(BOX_S)
print(NB)
print(intervel1)
print(intervel2)

[[14.95026657 -2.49513858  1.08329909 ... -0.77966473  1.33400379
   0.14598842]
 [12.91330806 -4.78882708  3.26570517 ...  0.97170965  2.42642645
  -0.18516359]
 [ 9.29097232  6.39343343  2.50659111 ...  5.98935651 -0.29082587
   2.321144  ]
 ...
 [ 1.38146831  3.9002878  -2.60232876 ...  0.62537818  0.69615654
   1.38614751]
 [ 5.78957577 -0.72839965  1.46103951 ... -3.22743447  0.26044708
  -0.85453244]
 [-0.0406802  -0.17041859 -2.06567524 ... -0.10295728 -0.22048968
   0.71882213]]
[[14.95026657 -2.49513858  1.08329909 ... -0.77966473  1.33400379
   0.14598842]
 [12.91330806 -4.78882708  3.26570517 ...  0.97170965  2.42642645
  -0.18516359]
 [ 9.29097232  6.39343343  2.50659111 ...  5.98935651 -0.29082587
   2.321144  ]
 ...
 [ 1.38146831  3.9002878  -2.60232876 ...  0.62537818  0.69615654
   1.38614751]
 [ 5.78957577 -0.72839965  1.46103951 ... -3.22743447  0.26044708
  -0.85453244]
 [-0.0406802  -0.17041859 -2.06567524 ... -0.10295728 -0.22048968
   0.71882213]]
[1 1 1 1 1 1 1 1

In [180]:
def Chessboard_online_merge_std(Box,BOX_miu,BOX_S,NB,intervel1,intervel2):
    threshold1=intervel1/2
    threshold2=intervel2/2
    NB1=0
    
    while NB1 != NB:
        CC = 0
        NB1 = NB
        for ii in range(NB):
            seq1 = [i for i in range(NB) if i != ii]

            distance1 = cdist(BOX_miu[ii].reshape(1,-1), BOX_miu[seq1], 'euclidean')
            distance2 = np.sqrt(cdist(BOX_miu[ii].reshape(1,-1), BOX_miu[seq1], 'cosine'))

            for jj in range(NB-1):
                if distance1[0,jj] < threshold1 and distance2[0,jj] < threshold2:
                    CC = 1
                    NB -= 1
                    Box = np.delete(Box, (ii), axis=0)
                    BOX_miu[seq1[jj]] = BOX_miu[seq1[jj]]*BOX_S[seq1[jj]]/(BOX_S[seq1[jj]]+BOX_S[ii])+BOX_miu[ii]*BOX_S[ii]/(BOX_S[seq1[jj]]+BOX_S[ii])
                    
                    BOX_S[seq1[jj]] = BOX_S[seq1[jj]] + BOX_S[ii]
                    BOX_miu = np.delete(BOX_miu, (ii), axis=0)
                    BOX_S = np.delete(BOX_S, (ii), axis=0)
                    
                    break
            if CC == 1:
                break                 
    return Box,BOX_miu,BOX_S,NB

In [181]:
%%time
Box_std, BOX_miu_std, BOX_S_std, NB_std = Chessboard_online_merge_std(Box,BOX_miu,BOX_S,NB,intervel1,intervel2)

Wall time: 121 ms


In [182]:
@njit
def Chessboard_online_merge_njit(Box,BOX_miu,BOX_S,NB,intervel1,intervel2):
    threshold1=intervel1/2
    threshold2=intervel2/2
    NB1=0
    
    L,W = BOX_miu.shape
    deleted_rows = 0
    while NB1 != NB:
        CC = 0
        NB1 = NB
        for ii in range(NB):
            
            seq1 = [i for i in range(NB) if i != ii]
            # distance1 = cdist(BOX_miu[ii].reshape(1,-1), BOX_miu[seq1], 'euclidean')
            
            distance1 = List()
            for i in range(NB):
                if i!= ii:
                    aux = 0
                    for jj in range(W):
                        aux += (BOX_miu[ii,jj] - BOX_miu[i,jj])**2
                
                    distance1.append(aux**.5)  
                        
                        
            #distance2 = np.sqrt(cdist(BOX_miu[ii].reshape(1,-1), BOX_miu[seq1], 'cosine'))
            
            distance2 = List()
            for i in range(NB):
                if i!= ii:
                    num = 0
                    den1 = 0
                    den2 = 0
                    for jj in range(W):
                        num += BOX_miu[ii,jj]*BOX_miu[i,jj]
                        den1 += BOX_miu[ii,jj]**2
                        den2 += BOX_miu[i,jj]**2
                    
                    d2 = (1 - num/(den1**.5 * den2**.5) )
                    if d2 < 0:               
                        distance2.append(d2**.5)  
                    else:
                        distance2.append(0)
            
            for jj in range(NB-1):
                if distance1[jj] < threshold1 and distance2[jj] < threshold2:
                    CC = 1
                    NB -= 1
                    #Box = np.delete(Box, (ii))
                    BOX_miu[seq1[jj]] = BOX_miu[seq1[jj]]*BOX_S[seq1[jj]]/(BOX_S[seq1[jj]]+BOX_S[ii])+BOX_miu[ii]*BOX_S[ii]/(BOX_S[seq1[jj]]+BOX_S[ii])
                    
                    BOX_S[seq1[jj]] = BOX_S[seq1[jj]] + BOX_S[ii]
                    
                    
                    ### ----------------------------------------------------------------------- ###
                    
                    
                    #BOX_miu = np.delete(BOX_miu, (ii))
                    #BOX_S = np.delete(BOX_S, (ii))
                    deleted_rows += 1
                    for i in range(L):
                        if i < ii:
                            for iii in range(W):
                                Box[i,iii] = Box[i,iii]
                                BOX_miu[i,iii] = BOX_miu[i,iii]
                            BOX_S[i] = BOX_S[i]
                        elif i < (L - deleted_rows):
                            for iii in range(W):
                                Box[i,iii] = Box[i+1,iii]
                                BOX_miu[i,iii] = BOX_miu[i+1,iii]
                            BOX_S[i] = BOX_S[i+1]
                        else:
                            for iii in range(W):
                                Box[i,iii] = 0
                                BOX_miu[i,iii] = 0
                            BOX_S[i] = 0
                            
                    
                    ### ----------------------------------------------------------------------- ###
                    break
            if CC == 1:
                break                 
    return Box,BOX_miu,BOX_S,NB, deleted_rows

In [183]:
%%time
Box_njit, BOX_miu_njit, BOX_S_njit, NB_njit, deleted_rows = Chessboard_online_merge_njit(Box,BOX_miu,BOX_S,NB,intervel1,intervel2)

Wall time: 1.09 s


In [184]:
print("----- Box -----")
print(sum(sum(Box_std - Box_njit)))
print("STD Type: ", type(Box_std))
print("NJIT Type: ", type(Box_njit))

print("----- Box_miu -----")
print(sum(sum(BOX_miu_std - BOX_miu_njit)))
print("STD Type: ", type(BOX_miu_std))
print("NJIT Type: ", type(BOX_miu_njit))

print("----- Box_S -----")
print(sum(BOX_S_std - BOX_S_njit))
print("STD Type: ", type(BOX_S_std))
print("NJIT Type: ", type(BOX_S_njit))

print("----- NB -----")
print(NB_std - NB_njit)
print("STD Type: ", type(NB_std))
print("NJIT Type: ", type(NB_njit))

----- Box -----
0.0
STD Type:  <class 'numpy.ndarray'>
NJIT Type:  <class 'numpy.ndarray'>
----- Box_miu -----
0.0
STD Type:  <class 'numpy.ndarray'>
NJIT Type:  <class 'numpy.ndarray'>
----- Box_S -----
0
STD Type:  <class 'numpy.ndarray'>
NJIT Type:  <class 'numpy.ndarray'>
----- NB -----
0
STD Type:  <class 'int'>
NJIT Type:  <class 'int'>


In [185]:
deleted_rows

0